In [4]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Store csv created in part one into a DataFrame
city_weather = pd.read_csv("../output_data/cities.csv")
city_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#store lat and lng into locations
locations = city_weather[['Lat', 'Lng']]

#get humidity
humidity = city_weather['Humidity']

In [4]:
# Heatmap of humidity
fig = gmaps.figure(center=(50.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,sakit,27.45,78.82,78.76,24,0,9.51,IN,1585764424
103,103,ormara,25.21,64.64,76.39,64,0,9.46,PK,1585764427
215,215,tura,25.52,90.22,72.75,23,0,3.09,IN,1585764441
274,274,chabahar,25.29,60.64,75.97,61,0,8.39,IR,1585764306
308,308,myitkyina,25.38,97.40,72.43,29,0,2.64,MM,1585764455
422,422,ghotki,28.01,69.32,77.00,61,0,5.70,PK,1585764470
484,484,nalut,30.33,10.85,73.78,16,0,6.08,LY,1585764477
544,544,sabha,27.04,14.43,78.24,16,0,9.01,LY,1585764485


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates


,City,Country,Lat,Lng,Hotel Name
81,sakit,IN,27.45,78.82,
103,ormara,PK,25.21,64.64,
215,tura,IN,25.52,90.22,
274,chabahar,IR,25.29,60.64,
308,myitkyina,MM,25.38,97.40,
422,ghotki,PK,28.01,69.32,
484,nalut,LY,30.33,10.85,
544,sabha,LY,27.04,14.43,


In [7]:
# Set parameters to search for a hotel


# Iterate through 

    # get lat, lng from df

    
    # Use the search term: "Hotel" and our lat/lng
   

    # make request and print url
    
    
    # convert to json
   
    
    # Grab the first hotel from the results and store the name




Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
81,sakit,IN,27.45,78.82,Laucky house
103,ormara,PK,25.21,64.64,
215,tura,IN,25.52,90.22,Hotel RIKMAN Continental
274,chabahar,IR,25.29,60.64,هتل گدروشیا
308,myitkyina,MM,25.38,97.40,Shwe Phyu Hotel
422,ghotki,PK,28.01,69.32,Mohalla Old Bazar
484,nalut,LY,30.33,10.85,
544,sabha,LY,27.04,14.43,محطة ام الارانب -سبها مواصلات


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map


# Display figure


Figure(layout=FigureLayout(height='420px'))